# 自然言語処理 ー 感情分析

## 1.　自然言語処理の6ステップ

#### ① 自然言語（文章）を用意
　吾輩は猫である。

#### ② 単語ごとに区切る（形態素解析）
　吾輩　／　は　／　猫　／　で　／　ある　／　。

#### ③ 助詞「「てにをは」などを削除する（データのクレンジング）
　吾輩　／　猫　　　※それ以外は削除

#### ④ 単語を原形（running→run など）に戻す（トークン化）
　今回はなし

#### ⑤ 各語句の頻出度を求める（BoW）
　吾輩：１　猫：1

#### ⑥ 各語句の重み（TF-IDF）を調べ、特徴語を決める
　吾輩：xxx　、猫：xxxx　　　→　特徴語は「猫」

#### ⑦ 他の文章との区別（分類）を行う（ロジスティック回帰）
　夏目漱石「吾輩は猫である」の特徴語：猫<br>
 　森鴎外「舞姫」の特徴語：姫
  
　どちらかの小説の一文を投入する　→　この小説は「夏目漱石」の小説です

## 2.　IMDbデータセットの取得<br>

今回は、以下のデータセットを使用する。

・[IMDb](http://ai.stanford.edu/~amaas/data/sentiment/)：映画レビューのデータセット

In [1]:
import os
import re
import sys
import tarfile
import time
import pyprind
import numpy as np
import pandas as pd
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('../input/movie-datacsv/movie_data.csv')
print(df.shape)

pd.set_option("display.max_colwidth", 150)
df.head(10)

(50000, 2)


,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve...",1
1,OK... so... I really like Kris Kristofferson and his usual easy going delivery of lines in his movies. Age has helped him with his soft spoken low...,0
2,"***SPOILER*** Do not read this, if you think about watching that movie, although it would be a waste of time. (By the way: The plot is so predicta...",0
3,hi for all the people who have seen this wonderful movie im sure thet you would have liked it as much as i. i love the songs once you have seen th...,1
4,"I recently bought the DVD, forgetting just how much I hated the movie version of ""A Chorus Line."" Every change the director Attenborough made to t...",0
5,Leave it to Braik to put on a good show. Finally he and Zorak are living their own lives outside of Spac Ghost Coast To Coast. I have to say that ...,1
6,"Nathan Detroit (Frank Sinatra) is the manager of the New York's longest- established floating craps game, and he needs $1000 to secure a new locat...",1
7,"To understand ""Crash Course"" in the right context, you must understand the 80's in TV. Most TV shows didn't have any point. The sitcom outpopulate...",1
8,"I've been impressed with Chavez's stance against globalisation for sometime now, but it wasn't until I saw the film at the Amsterdam documentary i...",1
9,This movie is directed by Renny Harlin the finnish miracle. Stallone is Gabe Walker. Cat and Mouse on the mountains with ruthless terrorists. Renn...,1


## 3.　BoWベクトル

文を数値化（ベクトル化）する方法として、**Bag of Words（BOW）**がある。

### 3.1　単語を特徴ベクトルに変換する

In [3]:
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, and one and one is two'])

scikit-learnに実装されているCountVectorizer()で、BoWベクトルをカウントする。

In [4]:
count = CountVectorizer()
bag = count.fit_transform(docs)

上記によって、次の3つの文章におけるBoWモデルが構築された。<br>
ここで、各単語に番号（インデックス）を付け、それぞれ確認をする。

In [5]:
print('単語:インデックス', count.vocabulary_)

単語:インデックス {'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


3文目の「the」の頻出度を調べてみる。<br>
0から数えるとインデックスは2となり、最後には単語を入れる。

In [6]:
print('頻出度：', bag.toarray()[2][count.vocabulary_['the']])

頻出度： 2


次に、特徴ベクトルを出力する。

In [7]:
pd.DataFrame(bag.toarray())

,0,1,2,3,4,5,6,7,8
0,0,1,0,1,1,0,1,0,0
1,0,1,0,0,0,1,1,0,1
2,2,3,2,1,1,1,2,1,1


行は1～3文目、列は各単語のインデックス番号を前提に表示されている。<br>
分かりやすいように、表示し直す。

In [8]:
pd.DataFrame(bag.toarray(),
             index=['1文目', '2文目', '3文目'],
             columns=['and', 'is', 'one', 'shining', 'sun', 'sweet', 'the', 'two', 'weather'])

,and,is,one,shining,sun,sweet,the,two,weather
1文目,0,1,0,1,1,0,1,0,0
2文目,0,1,0,0,0,1,1,0,1
3文目,2,3,2,1,1,1,2,1,1


### 3.2　TF-IDFを使って単語の関連性を評価する

$\rm{TF}$：対象の単語が、ある文書中にどれだけの頻度で出現しているか（頻度）<br>
$\rm{IDF}$：対象の単語が含まれる文が、文章全体にどれだけの頻度で出現していないか（珍しさ）<br>
$\rm{TF-IDF}$：Term Frequency – Inverse Document Frequency の略（重みづけの指標）

例えば、小説一冊が、下記で構成されているとする。

・$D$ 個の文<br>
・$N$ 個の単語

小説に単語 $x$ が $n$ 回現れるならば
$$\begin{eqnarray}
{\rm TF}=\frac{n}{N}
\end{eqnarray}$$
小説に単語 $x$ を含む文が $d$ 個あるならば
$$\begin{eqnarray}
{\rm IDF}=-\log_{10}\frac{d}{D}=\log_{10}\frac{D}{d}
\end{eqnarray}$$
よって、次のように求まる。
$$\begin{eqnarray}
{\rm TF-IDF}=\frac{n}{N}\log_{10}\frac{D}{d}
\end{eqnarray}$$

例題

太宰治の作品「一歩前進二歩退却」から一部を抜粋したものである。<br>
TF-IDFを求めなさい。

-----

　作家は、いよいよ窮屈である。何せ、眼光紙背に徹する読者ばかりを<br>
相手にしているのだから、うっかりできない。あんまり緊張して、ついには<br>
机のまえに端座したまま、そのまま、沈黙は金、という格言を底知れず肯定している。<br>
そんなあわれな作家さえ出て来ぬともかぎらない。<br>
　謙譲を、作家のみ要求し、作家は大いに恐縮し、卑屈なほどへりくだって<br>
そうして読者は旦那である。作家の私生活、底の底まで剥ごうとする。<br>
失敗である。安売りにしていいのは作品である。作家の人間までを売ってはいない。<br>
謙譲は、読者にこそ之を要求したい。

-----

下記をカウントした。<br>
<br>
単語数 $N$：150<br>
単語の種類：48<br>
文数 $D$：10

次に「作家」という単語 $x$ に着目する。

出現回数 $n$：6<br>
含まれる文数 $d$：5

よって、次のように求まる。<br>

$$\begin{eqnarray}
{\rm TF}=\frac{n}{N}=\frac{6}{150}=0.04
\end{eqnarray}$$

$$\begin{eqnarray}
{\rm IDF}=\log_{10}\frac{D}{d}=\log_{10}\frac{10}{5}=0.301
\end{eqnarray}$$

$$\begin{eqnarray}
{\rm TF-IDF}=0.0120
\end{eqnarray}$$<br>
<br>
実際の自然言語処理では、これを48種類の単語すべてに行っていく。

### 3.3　scikit-learnでTF-IDFを実装する

scikit-learnには、TfidTransformerという関数が実装されている。<br>
これは、fit_transformメソッドのCountVectorizerから「生の単語の出現頻度」を入力として受け取り、TF-IDFに変換する。

In [9]:
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
tfidf_vec = tfidf.fit_transform(count.fit_transform(docs)).toarray()

pd.DataFrame(tfidf_vec.round(2))

,0,1,2,3,4,5,6,7,8
0,0.0,0.43,0.0,0.56,0.56,0.00,0.43,0.00,0.00
1,0.0,0.43,0.0,0.00,0.00,0.56,0.43,0.00,0.56
2,0.5,0.45,0.5,0.19,0.19,0.19,0.30,0.25,0.19


smooth_idf：Truleがデフォルト<br>
use_idf：idf()の使用有無（idf()関数による重み付けを行うかどうか）<br>
norm：正則化の指定（デフォルトはなし、'l2'指定で単語ベクトルの長さが1になるよう正規化）<br>
toarray()：行列出力

それぞれの文や単語とも紐づけて可視化すると、以下のようになる。

In [10]:
tfidf_df = pd.DataFrame(tfidf_vec.round(2),
                        index=['1文目', '2文目', '3文目'],
                        columns=['and', 'is', 'one', 'shining', 'sun', 'sweet', 'the', 'two', 'weather'])

ここで、3つの文章を改めて見てみる。

・The sun is shining<br>
・The weather is sweet<br>
・The sun is shining, the weather is sweet, and one and one is two

「is」は3つの文章で使われているため、特徴語ではなく、どの文法に必要な語と判断される。<br>
TF-IDFが（0.45）とそれほど大きくないことからも考えられる。

一方で、「one」は3つ目の文章だけで2回使用されているので<br>
TF-IDFは（0.5）と少しだけ大きくなっている。

### 3.4　scikit-learnにおけるTF-IDFの定義式

また各々の特徴ベクトルのTF-IDFを「手動」で計算すると、上記の計算と合わないことが分かる。<br>
scikit-learnについて、厳密には下記の式で定義される。

$$\begin{eqnarray}
{\rm tf}(t,d)=N
\end{eqnarray}$$

$$\begin{eqnarray}
{\rm idf}(t,d) = \log\frac{1 + n_d}{1 + {\rm idf}(d, t)}+1
\end{eqnarray}$$

$$\begin{eqnarray}
{\rm tf-idf}(t,d)={\rm tf(t,d)}\times {\rm idf(t,d)}
\end{eqnarray}$$

$N$：1つの文章の中にある、単語$t$の出現回数

TF-IDFの正規化については、次式のように定義される。

$$v_{\text{norm}} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_{1}^{2} + v_{2}^{2} + \dots + v_{n}^{2}}} = \frac{v}{\big (\sum_{i=1}^{n} v_{i}^{2}\big)^\frac{1}{2}}$$

ここで、3つ目の文章について、改めて確認をする。

・The sun is shining, the weather is sweet, and one and one is two

今後は「is」に着目して、考えていくことにする。

### 3.5　scikit-learnの計算方法を手動で行う

TFについて、scikit-learnでは、単語「is」の出現回数がそのままの値となる。<br>
3つ目の文章において、「is」は3個あるため、次のようになる。

$$\text{tf}("is")= 3$$

$$\text{idf}("is", d3) = log \frac{1+3}{1+3}+1 = 0+1=1$$

よってTF-IDFは、次の値となる。

$$\text{tf-idf}("is",d3)= 3 \times 1 = 3$$

これらの計算を3つ目の文章の全ての単語に行った後で<br>
先ほど説明した正規化の式に従って、TF-IDFのL2正規化を行う。

$$\text{tf-idf}_{norm} = \frac{[3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0 , 1.69, 1.29]}{\sqrt{[3.39^2, 3.0^2, 3.39^2, 1.29^2, 1.29^2, 1.29^2, 2.0^2 , 1.69^2, 1.29^2]}}$$

$$=[0.5, 0.45, 0.5, 0.19, 0.19, 0.19, 0.3, 0.25, 0.19]$$

$$\Rightarrow \text{tf-idf}_{norm}("is", d3) = 0.45$$

In [11]:
tfidf_df.loc['3文目', 'is']

0.45

これにより、手動での計算結果とscikit-learnの計算結果の一致が証明できた。<br>
自然言語処理では、このTF・IDF・TF-IDFをもとに、特徴ベクトル（特徴語、重み）を決定する。

## 4.　クレンジング

まず、映画レビューデータセットの1つ目の文章から、最後の50文字を出力する。

In [12]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

出力結果を見てみると、不要な句読点や非英字文字が多い。<br>
感情分析に役に立ちそうな顔文字要素のある記号「　：)　」のみ残し、それ以外はすべて削除する。<br>
その際、今回はPythonの正規表現ライブラリを使用する。

In [13]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return text

re.sub：正規表現で指定した文字列を置換する<br>
str.lower(): すべての文字を小文字に変換

2行目：正規表現（<[^>]*>）を使用し、HTMLマークアップを削除<br>
3行目：顔文字を検索し、emoticonsに格納<br>
4行目：正規表現[\w]+を使って単語の一部でない文字を削除、小文字に変換し、emoticonsを加え、顔文字内の「-」を消去

In [14]:
print('変換前:', df.loc[0, 'review'][-50:])
print('変換後:', preprocessor(df.loc[0, 'review'][-50:]))

変換前: is seven.<br /><br />Title (Brazil): Not Available
変換後: is seven title brazil not available


In [15]:
print(preprocessor("</a>This :) is :( a test :-)!"))
print(preprocessor("!\/.i like ;.::python/:]/];/]"))
print(preprocessor('machine\::lear\[:::nig];@[/]'))

this is a test :) :( :)
 i like python 
machine lear nig 


再度、映画レビューデータセットの1行目を表示する。

In [16]:
print(df.shape)
df.head(10)

(50000, 2)


,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve...",1
1,OK... so... I really like Kris Kristofferson and his usual easy going delivery of lines in his movies. Age has helped him with his soft spoken low...,0
2,"***SPOILER*** Do not read this, if you think about watching that movie, although it would be a waste of time. (By the way: The plot is so predicta...",0
3,hi for all the people who have seen this wonderful movie im sure thet you would have liked it as much as i. i love the songs once you have seen th...,1
4,"I recently bought the DVD, forgetting just how much I hated the movie version of ""A Chorus Line."" Every change the director Attenborough made to t...",0
5,Leave it to Braik to put on a good show. Finally he and Zorak are living their own lives outside of Spac Ghost Coast To Coast. I have to say that ...,1
6,"Nathan Detroit (Frank Sinatra) is the manager of the New York's longest- established floating craps game, and he needs $1000 to secure a new locat...",1
7,"To understand ""Crash Course"" in the right context, you must understand the 80's in TV. Most TV shows didn't have any point. The sitcom outpopulate...",1
8,"I've been impressed with Chavez's stance against globalisation for sometime now, but it wasn't until I saw the film at the Amsterdam documentary i...",1
9,This movie is directed by Renny Harlin the finnish miracle. Stallone is Gabe Walker. Cat and Mouse on the mountains with ruthless terrorists. Renn...,1


これに、先ほど作成したpreprocessor関数を適用する。<br>
出力結果を確認すると、余計な文字や記号がなくなっていることが分かる。

In [17]:
df['review'] = df['review'].apply(preprocessor)
df.head(10)

,review,sentiment
0,in 1974 the teenager martha moxley maggie grace moves to the high class area of belle haven greenwich connecticut on the mischief night eve of hal...,1
1,ok so i really like kris kristofferson and his usual easy going delivery of lines in his movies age has helped him with his soft spoken low energy...,0
2,spoiler do not read this if you think about watching that movie although it would be a waste of time by the way the plot is so predictable that i...,0
3,hi for all the people who have seen this wonderful movie im sure thet you would have liked it as much as i i love the songs once you have seen the...,1
4,i recently bought the dvd forgetting just how much i hated the movie version of a chorus line every change the director attenborough made to the s...,0
5,leave it to braik to put on a good show finally he and zorak are living their own lives outside of spac ghost coast to coast i have to say that i ...,1
6,nathan detroit frank sinatra is the manager of the new york s longest established floating craps game and he needs 1000 to secure a new location c...,1
7,to understand crash course in the right context you must understand the 80 s in tv most tv shows didn t have any point the sitcom outpopulated the...,1
8,i ve been impressed with chavez s stance against globalisation for sometime now but it wasn t until i saw the film at the amsterdam documentary in...,1
9,this movie is directed by renny harlin the finnish miracle stallone is gabe walker cat and mouse on the mountains with ruthless terrorists renny h...,1


## 5.　トークン化

トークン化：文章を個々の単語に分割したり、変換したりすること<br>
ワードステミング：単語を原形に変換すること　（例）running →　run　など

ワードステミングは、PorterStemmerによって開発され、Porter stemmingアルゴリズムとも呼ばれる。<br>
NLTKライブラリに実装されている。

In [18]:
porter = PorterStemmer()

def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

split()：文字を空白で分割<br>
tokenizer_porter(text)：split()で分割した単語をfor文で1個ずつ取り出し、ワードステミングを実行

ここで、トークン化とトークン化＋ワードワードステミングの結果を比較する。

In [19]:
print(tokenizer('runners like running and thus they run'))
print(tokenizer_porter('runners like running and thus they run'))

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']
['runner', 'like', 'run', 'and', 'thu', 'they', 'run']


また、ストップワードの除去も行う。<br>
こちらも、NLTKライブラリで実行することができる。

まず、nltkに登録されているストップワードをダウンロードする。<br>
その後、ストップワードでないものにtokenizer_porter()を適用し、単語wを抽出する。

In [20]:
nltk.download('stopwords')

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['runner', 'like', 'run', 'run', 'lot']

## 6.　ロジスティック回帰による文書の分類

ロジスティック回帰を用いて、下記に分類する。

1：肯定的なレビュー（面白かった など）<br>
0：否定的なレビュー（つまなかった など）

まず、クレンジングしたテキストのDataframeを、訓練データとテストデータに分割する。

In [21]:
print(df.shape)
df.head(10)

(50000, 2)


,review,sentiment
0,in 1974 the teenager martha moxley maggie grace moves to the high class area of belle haven greenwich connecticut on the mischief night eve of hal...,1
1,ok so i really like kris kristofferson and his usual easy going delivery of lines in his movies age has helped him with his soft spoken low energy...,0
2,spoiler do not read this if you think about watching that movie although it would be a waste of time by the way the plot is so predictable that i...,0
3,hi for all the people who have seen this wonderful movie im sure thet you would have liked it as much as i i love the songs once you have seen the...,1
4,i recently bought the dvd forgetting just how much i hated the movie version of a chorus line every change the director attenborough made to the s...,0
5,leave it to braik to put on a good show finally he and zorak are living their own lives outside of spac ghost coast to coast i have to say that i ...,1
6,nathan detroit frank sinatra is the manager of the new york s longest established floating craps game and he needs 1000 to secure a new location c...,1
7,to understand crash course in the right context you must understand the 80 s in tv most tv shows didn t have any point the sitcom outpopulated the...,1
8,i ve been impressed with chavez s stance against globalisation for sometime now but it wasn t until i saw the film at the amsterdam documentary in...,1
9,this movie is directed by renny harlin the finnish miracle stallone is gabe walker cat and mouse on the mountains with ruthless terrorists renny h...,1


In [22]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

次に、GridSearchCVを使って、最適なパラメーター集合を求める。<br>
ここでは、5分割交差検証を使用する。

In [23]:
param_grid = [
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
    
              ]

Pipelineで、TfidfVectorizer()とLogisticRegression()をセットにする。

In [24]:
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
logreg = LogisticRegression(random_state=0)

lr_tfidf = Pipeline([('vect', tfidf), ('clf', logreg)])

In [25]:
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

scoring：評価指標（初期値はNone、分類なら正解率、回帰ならMSE など）<br>
verbose：途中経過を詳細に出力（初期値はTrue）<br>
n_jobs：並列実行数（初期値は1）

In [26]:
if 'TRAVIS' in os.environ:
    gs_lr_tfidf.verbose=2

    X_train = df.loc[:250, 'review'].values
    y_train = df.loc[:250, 'sentiment'].values
    X_test = df.loc[25000:25250, 'review'].values
    y_test = df.loc[25000:25250, 'sentiment'].values

verbose: 0は標準出力にログを出力しない、1はログをプログレスバーで標準出力，2はエポックごとに1行のログを出力

In [27]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 36.3min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 149.6min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 184.6min finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd",...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
           

グリッドサーチが完了した後、性能指標が最も高くなるパラメータセットを出力する。

In [28]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

Best parameter set: {'clf__C': 1.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each',

In [29]:
print(gs_lr_tfidf.best_estimator_)

Pipeline(steps=[('vect',
                 TfidfVectorizer(lowercase=False,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...],
                                 tokenizer=<function tokenizer at 0x7fd48d6a4680>)),
                ('clf', LogisticRegression(random_state=0))])


In [30]:
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

CV Accuracy: 0.889


よって、「C=10のときに、ロジスティック回帰とTF-IDFを組み合わせることで、性能指標が最も高くなる」ことが分かった。

In [31]:
clf = gs_lr_tfidf.best_estimator_

y_pred = clf.predict(X_test)
print(y_pred[:10])

[0 1 1 0 0 1 0 1 1 0]


In [32]:
df = pd.DataFrame(X_test, columns=['review'])
df['prediction'] = y_pred
df['answer'] = y_test

df.head(10)

,review,prediction,answer
0,there s a part of me that would like to give this movie a high rating considering that it was made in 1953 this is a very courageous movie about t...,0,0
1,excellent and moving story of the end of a uniquely intimate affair then again the point of the film to paraphrase another comment is that every r...,1,1
2,surprisingly well acted well written movie about hard rockin but decent young man getting that much hoped for ticket to stardom his favorite heavy...,1,1
3,what garbage is there actually no part ii if this movie actually ends the way it did everyone involved with this movie should be ashamed this movi...,0,0
4,basically this was obviously designed to be promotional material for the movie produced by the same horrible director which happens to be even wor...,0,0
5,robot jox doesn t suffer from story or bad effects i mean this was 1990 if you know what i m talking about robocop 2 still used the stop animation...,1,1
6,i m sorry to say this but the acting in this film is horrible the dialogue sounds as if they are reading their lines for the first time ever perha...,0,0
7,greystoke the legend of tarzan lord of the apes is based on the classic book tarzan of the apes by edgar rice burroughs and is a more faithful ada...,1,1
8,i searched for this movie for years apparently it ain t available here in the states so bought me a copy off ebay four young hunters and three of ...,1,1
9,once again disney manages to make a children s movie which totally ignores its background about the only thing common with this and the original g...,0,0


In [33]:
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.895


今回作成した機械学習モデルでは、映画レビューのデータセットを投入したときに<br>
各レビューが肯定的か否定的なのかを、約90%の精度で予測できることが分かった。